In [1]:
import pandas as pd
import numpy as np

In [59]:
df = pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB


In [60]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)
df.shape

(891, 9)

In [61]:
from scipy.stats import mode

mode_embarked=df['Embarked'].value_counts().index[0]
df["Embarked"] = df["Embarked"].fillna(mode_embarked)
df.shape

(891, 9)

In [10]:
df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)

In [16]:
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

In [18]:
df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]

In [20]:
train_data = df.values

In [43]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100)
model = model.fit(train_data[0:,2:],train_data[0:,0])

In [45]:
df_test = pd.read_csv('test.csv')

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)

fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x:
                            fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)

In [47]:
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],
                axis=1)

In [66]:
#df_test = df_test.drop(['Sex', 'Embarked'], axis=1)
df_test.shape

(418, 10)

In [64]:
test_data = df_test.values

output = model.predict(test_data[:,1:])
test_data.shape

(418, 10)

In [63]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]
result.shape

(418, 2)

In [54]:
df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])

In [56]:
df_result.shape

(418, 2)

In [ ]:
df_result.to_csv('results/titanic_1-2.csv',index=False)